In [1]:
import torch
torch.cuda.empty_cache()

Mounting the google drive and making sure we have the right stuff!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Image Domain"
%ls

Mounted at /content/drive
/content/drive/My Drive/Image Domain
BasicEncoderRun.ipynb  InDomain/   TestEncoderRun.ipynb
Cavity_images/         OutDomain/  TrainingImages/


Importing the libraries!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import glob

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import torch
import torchvision
from torchvision import transforms, datasets

Checking if we have GPU access!

In [3]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Device we are running on is", device)

Device we are running on is mps


Preprocessing the images to have a size of 1024, 1024 for now because I don't know what else to do. Also loading the training images as a tensor.

TO DO: Make this into a function which I can call again later

In [ ]:
TRAIN_DOMAIN_IMAGE_PATH = '/content/drive/My Drive/Image Domain/Cavity_images/cavity_nome_underfocus_only/train'
IN_DOMAIN_IMAGE_PATH = '/content/drive/My Drive/Image Domain/Cavity_images/cavity_nome_overfocus_only/val'
OUT_DOMAIN_IMAGE_PATH = '/contentdrive/My Drive/Image Domain/Cavity_images/cavity_nome_underfocus_only/val'

compression_vector_size = 64
image_list = []
image_vector = np.empty((95, 1024, 1024), dtype=float)
desired_height = 1024
desired_width = 1024
for filename in glob.glob(TRAIN_DOMAIN_IMAGE_PATH + "/*.png"):
    im=Image.open(filename)
    transform = transforms.Compose([
      transforms.Resize((desired_height, desired_width)),
      transforms.ToTensor()
    ])
    transformed_im = transform(im)
    image_list.append(transformed_im)
num_images = len(image_list)
num_channels , height, width = image_list[0].shape
image_tensor = torch.zeros((num_images, num_channels, height, width))
for i, image in enumerate(image_list):
  if(image.size() == (3, 1024, 1024)):
    image_tensor[i, 0, :, :] = torch.mean(image, dim=0)
  else:
    image_tensor[i, :, :, :] = image
print(image_tensor.size())
Image_load = torch.utils.data.DataLoader(image_tensor, batch_size=16, shuffle=True)

torch.Size([95, 1, 1024, 1024])


Making the architecture for the autoencoder

In [ ]:
class Autoencoder(torch.nn.Module):
  def __init__(self):
    super().__init__()
    # Add more layers, Conv 2D Layers
    self.encoder = torch.nn.Sequential(
        torch.nn.Linear(1024*1024, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 128)
    )
    self.decoder = torch.nn.Sequential(
        torch.nn.Linear(128, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 1024*1024)
    )

  def forward(self, x):
    encoded = self.encoder(x)
    # Look at the distribution of histograms, make 128 bins for the points and
    # then see what it looks like
    decoded = self.decoder(encoded)
    return decoded, encoded
'''
  def __init__(self):
    super().__init__()

    self.encoder = torch.nn.Sequential(
        torch.nn.Linear(1024*1024, 16384),
        torch.nn.ReLU(),
        torch.nn.Linear(16384, 4096),
        torch.nn.ReLU(),
        torch.nn.Linear(4096, 1024),
        torch.nn.ReLU(),
        torch.nn.Linear(1024, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 18),
        torch.nn.ReLU(),
        torch.nn.Linear(18, 9)
    )

    self.decoder = torch.nn.Sequential(
        torch.nn.Linear(9, 18),
        torch.nn.ReLU(),
        torch.nn.Linear(18, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 1024),
        torch.nn.ReLU(),
        torch.nn.Linear(1024, 4096),
        torch.nn.ReLU(),
        torch.nn.Linear(4096, 16384),
        torch.nn.ReLU(),
        torch.nn.Linear(16384, 1024*1024),
        torch.nn.Sigmoid()
    )

  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
'''

'\n  def __init__(self):\n    super().__init__()\n    \n    self.encoder = torch.nn.Sequential(\n        torch.nn.Linear(1024*1024, 16384),\n        torch.nn.ReLU(),\n        torch.nn.Linear(16384, 4096),\n        torch.nn.ReLU(),\n        torch.nn.Linear(4096, 1024),\n        torch.nn.ReLU(),\n        torch.nn.Linear(1024, 256),\n        torch.nn.ReLU(),\n        torch.nn.Linear(256, 64),\n        torch.nn.ReLU(),\n        torch.nn.Linear(64, 18),\n        torch.nn.ReLU(),\n        torch.nn.Linear(18, 9)\n    )\n\n    self.decoder = torch.nn.Sequential(\n        torch.nn.Linear(9, 18),\n        torch.nn.ReLU(),\n        torch.nn.Linear(18, 64),\n        torch.nn.ReLU(),\n        torch.nn.Linear(64, 256),\n        torch.nn.ReLU(),\n        torch.nn.Linear(256, 1024),\n        torch.nn.ReLU(),\n        torch.nn.Linear(1024, 4096),\n        torch.nn.ReLU(),\n        torch.nn.Linear(4096, 16384),\n        torch.nn.ReLU(),\n        torch.nn.Linear(16384, 1024*1024),\n        torch.nn.Sigmo

In [ ]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = Autoencoder().to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = torch.nn.MSELoss()
'''
model = Autoencoder()
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-1, weight_decay = 1e8)
'''

In [ ]:
epochs = 10
outputs = []
loss = 0
for epoch in range(epochs):
  for batch_features in Image_load:
    batch_features = batch_features.view(-1, 1024*1024).to(device)
    optimizer.zero_grad()
    #Images created
    outputs = model(batch_features)
    # compute training reconstruction loss
    train_loss = criterion(outputs, batch_features)
    # compute accumulated gradients
    train_loss.backward()
    # perform parameter update based on current gradients
    optimizer.step()
    # add the mini-batch training loss to epoch loss
    loss += train_loss.item()

  loss = loss / len(Image_load)
  print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

AttributeError: ignored

In [ ]:
plt.gray()
for i, item in enumerate(batch_features):
   print("Original Images!")
   item = item.reshape((-1, 1024, 1024))
   plt.imshow(item[0].cpu())
   plt.show()
for i, item in enumerate(outputs):
   print("Reconstructed Images!")
   item = item.reshape((-1, 1024, 1024))
   plt.imshow(item[0].cpu().detach().numpy())
   plt.show()

Output hidden; open in https://colab.research.google.com to view.